In [ ]:
!pip install qiskit

In [2]:
from google.colab import files
uploaded = files.upload()

Saving data2.csv to data2.csv


In [3]:
import numpy as np

from qiskit import BasicAer
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.aqua.algorithms import VQC
from qiskit.aqua.components.optimizers import SPSA
from qiskit.circuit.library import TwoLocal, ZZFeatureMap
from qiskit.aqua.utils import split_dataset_to_data_and_labels, map_label_to_class_name



In [4]:
seed = 10600
aqua_globals.random_seed = seed

In [5]:
import io
import pandas as pd
dat = pd.read_csv(io.BytesIO(uploaded['data2.csv']))

In [6]:
dat

,ZIP Code,Case rate(weekly),Test Rate(weekly),Safe(1)/Unsafe(0)
0,60608,50,2294,0
1,60613,48,3195,1
2,60614,83,4624,0
3,60617,27,1364,0
4,60618,72,2140,0
5,60619,41,1801,0
6,60622,66,3055,1
7,60623,45,1478,0
8,60632,57,1321,0
9,60633,86,1802,0


In [7]:
safe = []
unsafe = []
for i in range(len(dat)):
    if dat['Safe(1)/Unsafe(0)'].values[i] == 1:
        safe.append([dat['Case rate(weekly)'].values[i],dat['Test Rate(weekly)'].values[i]])
    else: 
        unsafe.append([dat['Case rate(weekly)'].values[i],dat['Test Rate(weekly)'].values[i]])

In [24]:
feature_dim = 2
training_input = {'safe' : np.array(safe[0:10]), 'unsafe' : np.array(unsafe[0:38])} #change from 40 to 20
test_input = {'safe' : np.array(safe[10:-1]), 'unsafe' : np.array(unsafe[38:-1])} #change from 40 to 50
class_labels = {'unsafe', 'safe'}

In [25]:
onearr = [1]*len(safe)
zeroarr = [0]*len(unsafe)
label = np.concatenate((onearr,zeroarr))
print(label)

[1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [26]:
join = safe + unsafe
print(join)

[[48, 3195], [66, 3055], [54, 3038], [30, 4900], [68, 3946], [32, 5063], [50, 4506], [50, 6354], [50, 3216], [68, 5089], [32, 8090], [55, 4981], [50, 2294], [83, 4624], [27, 1364], [72, 2140], [41, 1801], [45, 1478], [57, 1321], [86, 1802], [114, 2025], [44, 1578], [71, 1539], [102, 1695], [74, 2564], [118, 2694], [55, 2474], [59, 2649], [89, 4118], [76, 1820], [90, 2267], [62, 2578], [24, 1632], [69, 2794], [71, 2656], [53, 2859], [161, 2974], [85, 3629], [64, 1754], [71, 2956], [137, 2156], [61, 2228], [28, 2420], [36, 1542], [128, 8184], [85, 1345], [94, 2097], [76, 3190], [73, 1802], [55, 2334], [63, 2236], [66, 2110], [69, 1760], [66, 1963], [101, 3687]]


In [27]:
datapoints = [np.array(join),label]

In [53]:
feature_map = ZZFeatureMap(feature_dimension=feature_dim, reps=2, entanglement='linear')
optimizer = SPSA(maxiter=40, c0=4.0, skip_calibration=True)
var_form = TwoLocal(feature_dim, ['ry', 'rz'], 'cz', reps=3)
vqc = VQC(optimizer, feature_map, var_form, training_input, test_input, datapoints[0])

backend = BasicAer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=seed, seed_transpiler=seed)

result = vqc.run(quantum_instance)

print(f'Testing success ratio: {result["testing_accuracy"]}')
print()
print('Prediction from datapoints set:')
print(f'  ground truth: {map_label_to_class_name(datapoints[1], vqc.label_to_class)}')
print(f'  prediction:   {result["predicted_classes"]}')
predicted_labels = result["predicted_labels"]
print(f'  success rate: {100*np.count_nonzero(predicted_labels == datapoints[1])/len(predicted_labels)}%')

Testing success ratio: 0.6

Prediction from datapoints set:
  ground truth: ['unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe']
  prediction:   ['unsafe', 'unsafe', 'safe', 'unsafe', 'safe', 'unsafe', 'unsafe', 'safe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'safe', 'safe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'safe', 'unsafe', 'safe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'safe', 'safe', 'unsafe', 'safe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'safe', 'safe', 'unsafe', 'unsaf

In [39]:
from qiskit.circuit.library import ZFeatureMap

In [51]:
feature_map = ZFeatureMap(feature_dimension=feature_dim, reps=4)
optimizer = SPSA(maxiter=40, c0=4.0, skip_calibration=True)
var_form = TwoLocal(feature_dim, ['ry', 'rz'], 'cz', reps=3)
vqc = VQC(optimizer, feature_map, var_form, training_input, test_input, datapoints[0])

backend = BasicAer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=seed, seed_transpiler=seed)

result = vqc.run(quantum_instance)

print(f'Testing success ratio: {result["testing_accuracy"]}')
print()
print('Prediction from datapoints set:')
print(f'  ground truth: {map_label_to_class_name(datapoints[1], vqc.label_to_class)}')
print(f'  prediction:   {result["predicted_classes"]}')
predicted_labels = result["predicted_labels"]
print(f'  success rate: {100*np.count_nonzero(predicted_labels == datapoints[1])/len(predicted_labels)}%')

Testing success ratio: 0.6

Prediction from datapoints set:
  ground truth: ['unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe']
  prediction:   ['unsafe', 'unsafe', 'unsafe', 'safe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'safe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'safe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe'

In [47]:
from qiskit.circuit.library import PauliFeatureMap

In [55]:
feature_map = PauliFeatureMap(feature_dimension=feature_dim, reps=3, paulis= ['Z','X','ZY'])
optimizer = SPSA(maxiter=40, c0=4.0, skip_calibration=True)
var_form = TwoLocal(feature_dim, ['ry', 'rz'], 'cz', reps=3)
vqc = VQC(optimizer, feature_map, var_form, training_input, test_input, datapoints[0])

backend = BasicAer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=seed, seed_transpiler=seed)

result = vqc.run(quantum_instance)

print(f'Testing success ratio: {result["testing_accuracy"]}')
print()
print('Prediction from datapoints set:')
print(f'  ground truth: {map_label_to_class_name(datapoints[1], vqc.label_to_class)}')
print(f'  prediction:   {result["predicted_classes"]}')
predicted_labels = result["predicted_labels"]
print(f'  success rate: {100*np.count_nonzero(predicted_labels == datapoints[1])/len(predicted_labels)}%')

Testing success ratio: 0.8

Prediction from datapoints set:
  ground truth: ['unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe']
  prediction:   ['unsafe', 'unsafe', 'safe', 'safe', 'safe', 'unsafe', 'safe', 'safe', 'safe', 'unsafe', 'unsafe', 'unsafe', 'safe', 'unsafe', 'unsafe', 'safe', 'safe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'safe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'safe', 'unsafe', 'unsafe', 'unsafe', 'safe', 'unsafe', 'unsafe', 'unsafe', 'safe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'safe', 